## 1. title에 대한 TF-IDF 생성

In [63]:
# 한글 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# data 폴더 설정
fdir='/content/drive/MyDrive/Notability/Github/Python_Machine_Learning/etc/'

In [66]:
# 필요 모듈 설치
!pip install konlpy
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-01-19 07:47:01--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-01-19 07:47:01--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2563821243fa15c40b3d38e9b0.dl.dropboxusercontent.com/cd/0/get/B00Fzj5SyBfeHIj-qUqKQC1M4fPQhVpy_Hxh8ZrqvVDkDW3Tv3jtQ2lXxXP9oJ4YUIyJmMerYf-0NxmS5eZpnvOP4iQFkAVDH1S04GBul2KXKmH6kEa-bPqmgmX3NO068fnL3MZj_30f1gWHB960TwCfKCyvW1TZBR3Gkpt_Em1LQnfuFWYlEZiuJjH3bT4hU80/file?dl=1# [following]
--2023-01-

In [67]:
from konlpy.tag import Okt, Mecab
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re
import pickle

plt.rc('font', family='NanumBarunGothic') 

import warnings
warnings.filterwarnings(action='ignore')

In [68]:
# 데이터 가져오기
df=pd.read_csv(fdir+'부산관광지.csv')
df.head()

,ques_date,title,question,answer
0,2021.03.24,부산 관광지,제가 다음주에 부산으로 여행을 가는데 가볼만한 관광지 추천좀 해주세요!!!,부산하면바다지요.부산 해운대관광열차타고 바다위를 한바퀴 쭉 돌다보면~속이 ...
1,끌올 2022.12.19,"부산 워크숍 관광지, 식당",안녕하세요~! 23년 2월에 부산으로 워크숍을 가게 됐습니다.인원은 25명 정도이...,"호텔셔틀 여부 물어보고아니면 부산,울산소재 관광버스회사에 사전 예약하면 됩..."
2,끌올 2022.12.19,"부산 워크숍 관광지, 식당",안녕하세요~! 23년 2월에 부산으로 워크숍을 가게 됐습니다.인원은 25명 정도이...,부산 워크샵 버스대절 알아보고 계심 여기서 견적 한번 받아보시는거 어떠세요...
3,2021.05.16,부산 관광지 소개하기,"중학교 1학년인데요 사회 수행평가로 부산의 관광지 3개, 음식 3개, 숙박1개, ...","안녕하세요 마음을 전달하는 시바 입니다 ლ(╹◡╹ლ) ​해운대, 광안리, ..."
4,2021.03.14,부산 관광지 순위,해운대가 1등?아님광안리가 1등?뭐가 사람들이 제일 좋아하는 관광지인가요?,외국인이 한창많올때부산의관광지순위가 서면이1위 그다음 해운대 광안리 남포동...


In [69]:
## 'question'가 'title'과 동일한 경우에 공백으로 변경
for i in df[df['question']==df['title']].index:
  df.loc[i,'question']= ' '

df[df['question']==df['title']]

,ques_date,title,question,answer


In [70]:
# 'title'과'question'을 합쳐서 'title_question' 필드 생성
df['title_question'] = df['title'] + df['question']
df.head()

,ques_date,title,question,answer,title_question
0,2021.03.24,부산 관광지,제가 다음주에 부산으로 여행을 가는데 가볼만한 관광지 추천좀 해주세요!!!,부산하면바다지요.부산 해운대관광열차타고 바다위를 한바퀴 쭉 돌다보면~속이 ...,부산 관광지 제가 다음주에 부산으로 여행을 가는데 가볼만한 관광지 추...
1,끌올 2022.12.19,"부산 워크숍 관광지, 식당",안녕하세요~! 23년 2월에 부산으로 워크숍을 가게 됐습니다.인원은 25명 정도이...,"호텔셔틀 여부 물어보고아니면 부산,울산소재 관광버스회사에 사전 예약하면 됩...","부산 워크숍 관광지, 식당 안녕하세요~! 23년 2월에 부산으로 워크..."
2,끌올 2022.12.19,"부산 워크숍 관광지, 식당",안녕하세요~! 23년 2월에 부산으로 워크숍을 가게 됐습니다.인원은 25명 정도이...,부산 워크샵 버스대절 알아보고 계심 여기서 견적 한번 받아보시는거 어떠세요...,"부산 워크숍 관광지, 식당 안녕하세요~! 23년 2월에 부산으로 워크..."
3,2021.05.16,부산 관광지 소개하기,"중학교 1학년인데요 사회 수행평가로 부산의 관광지 3개, 음식 3개, 숙박1개, ...","안녕하세요 마음을 전달하는 시바 입니다 ლ(╹◡╹ლ) ​해운대, 광안리, ...",부산 관광지 소개하기 중학교 1학년인데요 사회 수행평가로 부산의 관광...
4,2021.03.14,부산 관광지 순위,해운대가 1등?아님광안리가 1등?뭐가 사람들이 제일 좋아하는 관광지인가요?,외국인이 한창많올때부산의관광지순위가 서면이1위 그다음 해운대 광안리 남포동...,부산 관광지 순위 해운대가 1등?아님광안리가 1등?뭐가 사람들이 제일...


In [71]:
# 데이터 저장
df.to_csv(fdir+'부산관광지_up.csv',index=False)

1. 데이터 단어추출(명사)

In [72]:
# title에서 명사(nouns) 추출
mecab = Mecab()
title_nouns=[mecab.nouns(i) for i in df['title']]

with open(fdir+'titel_nouns.pickle', 'wb') as f:
    pickle.dump(title_nouns, f, pickle.HIGHEST_PROTOCOL)

print(title_nouns[:3])

[['부산', '관광지'], ['부산', '워크숍', '관광지', '식당'], ['부산', '워크숍', '관광지', '식당']]


In [73]:
# question에서 명사(nouns) 추출
mecab = Mecab()
question_nouns=[mecab.nouns(i) for i in df['question']]

with open(fdir+'question_nouns.pickle', 'wb') as f:
    pickle.dump(question_nouns, f, pickle.HIGHEST_PROTOCOL)

print(question_nouns[:3])

[['제', '다음', '주', '부산', '여행', '만', '관광지', '추천'], ['안녕', '년', '월', '부산', '워크숍', '인원', '명', '정도', '이용', '도착', '후', '버스', '대절', '생각', '중', '버스', '업체', '관광지', '단체', '이용', '수', '식당', '추천', '감사', '관광지', '곳', '장소', '줄', '모두', '최대한', '만족', '생각', '구성원', '대', '중', '후반', '액티비티', '것', '것', '날씨', '걱정', '요트', '호텔', '게'], ['안녕', '년', '월', '부산', '워크숍', '인원', '명', '정도', '이용', '도착', '후', '버스', '대절', '생각', '중', '버스', '업체', '관광지', '단체', '이용', '수', '식당', '추천', '감사', '관광지', '곳', '장소', '줄', '모두', '최대한', '만족', '생각', '구성원', '대', '중', '후반', '액티비티', '것', '것', '날씨', '걱정', '요트', '호텔', '게']]


### title에 대한 TF-IDF

In [74]:
# 단어에 대한 고유 목록 생성
word_list=[]
for wl in title_nouns:
  for w in wl:
    word_list.append(w)

title_vocab=sorted(list(set(word_list)))
print(title_vocab)

['가요', '가족', '가지', '갈라', '갈만', '감', '감사', '감안', '강남', '개', '개념', '개미집', '개월', '거', '거기', '거리', '거제', '건', '건데', '걸', '것', '게', '게스트', '겨울', '경남', '경로', '경비', '경영학', '경제', '경주', '경험', '계획', '고등학교', '고등학생', '고딩', '고민', '고요', '곳', '공원', '공항', '관광', '관광지', '관련', '광시', '광안', '광안리', '광주', '교육', '교통', '교통수단', '교통편', '구경', '구군', '구요', '국내', '국립', '국어', '군대', '군인', '궁사', '그때', '그리핀', '그쪽', '근처', '금빛', '급', '기대', '기장', '기준', '기차', '기행문', '길', '김해', '깡통', '껀', '끝', '나', '난이도', '날', '남자', '남편', '남포동', '낮', '내공', '놀러', '다음', '단계', '달맞이', '답변자', '당일', '당일치기', '대교', '대구', '대역', '대전', '대중교통', '대천', '데', '데이트', '덴데', '도움', '도착', '동기', '동백섬', '동선', '동용', '돼지국밥', '둘', '뒤', '드라마', '등', '등등', '딸', '때', '루트', '리조트', '마사지', '마을', '만', '말', '말씀', '맛집', '먹거리', '먹방', '명', '명물', '명소', '모래', '모습', '모자', '몬드', '문의', '문화', '물품', '미성년자', '미자', '밀면', '바다', '바닷가', '박', '박물관', '밤', '밤바다', '버스', '번', '베이', '보관', '보관함', '보수동', '본전', '볼거리', '부', '부근', '부모', '부산', '부산관', '부산광역시', '부산역', '부천', '부탁', '부평', '분', '비', '비용', '비

In [75]:
# 한글자음/모음, 한자, 특수기호 등 제거
tmp=[]
for w in title_vocab:
    tmp.append(re.findall('[a-zA-Z가-힣0-9]+', w))
print(tmp)
#print("".join(tmp)) -> error

temps=[]
for idx in tmp:
  temps.append("".join(idx))
print(temps)

[['가요'], ['가족'], ['가지'], ['갈라'], ['갈만'], ['감'], ['감사'], ['감안'], ['강남'], ['개'], ['개념'], ['개미집'], ['개월'], ['거'], ['거기'], ['거리'], ['거제'], ['건'], ['건데'], ['걸'], ['것'], ['게'], ['게스트'], ['겨울'], ['경남'], ['경로'], ['경비'], ['경영학'], ['경제'], ['경주'], ['경험'], ['계획'], ['고등학교'], ['고등학생'], ['고딩'], ['고민'], ['고요'], ['곳'], ['공원'], ['공항'], ['관광'], ['관광지'], ['관련'], ['광시'], ['광안'], ['광안리'], ['광주'], ['교육'], ['교통'], ['교통수단'], ['교통편'], ['구경'], ['구군'], ['구요'], ['국내'], ['국립'], ['국어'], ['군대'], ['군인'], ['궁사'], ['그때'], ['그리핀'], ['그쪽'], ['근처'], ['금빛'], ['급'], ['기대'], ['기장'], ['기준'], ['기차'], ['기행문'], ['길'], ['김해'], ['깡통'], ['껀'], ['끝'], ['나'], ['난이도'], ['날'], ['남자'], ['남편'], ['남포동'], ['낮'], ['내공'], ['놀러'], ['다음'], ['단계'], ['달맞이'], ['답변자'], ['당일'], ['당일치기'], ['대교'], ['대구'], ['대역'], ['대전'], ['대중교통'], ['대천'], ['데'], ['데이트'], ['덴데'], ['도움'], ['도착'], ['동기'], ['동백섬'], ['동선'], ['동용'], ['돼지국밥'], ['둘'], ['뒤'], ['드라마'], ['등'], ['등등'], ['딸'], ['때'], ['루트'], ['리조트'], ['마사지'], ['마을'], ['만'], ['말'], ['말씀'], ['맛집'], ['먹거리'], ['먹방'], 

In [76]:
tmp=["".join(re.findall('[a-zA-Z가-힣0-9]+',w)) for w in title_vocab]
vocab=[w for w in sorted(list(set(tmp))) if w != '']
vocab[:10]

['가요', '가족', '가지', '갈라', '갈만', '감', '감사', '감안', '강남', '개']

In [77]:
# TF 계산
tf_lst=[]
for nrow in title_nouns:
  tf_lst.append([])
  for voc in vocab:
    tf_lst[-1].append(nrow.count(voc))

tf=pd.DataFrame(tf_lst,columns=vocab)
tf

,가요,가족,가지,갈라,갈만,감,감사,감안,강남,개,...,화요일,환자,활동,회,회사,횡인,후,후임,휴가,힐링
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# tf 데이터 pickle로 저장
with open(fdir+'title_tf.pickle','wb') as f:
    pickle.dump(tf, f, pickle.HIGHEST_PROTOCOL)

In [79]:
# IDF 생성
D = len(tf)
df_=tf.astype(bool).sum() # 단어 단위의 빈도수
idf=np.log10(D/(1+df_))
idf

가요    1.705130
가족    1.584087
가지    2.307190
갈라    2.307190
갈만    1.675167
        ...   
횡인    2.374137
후     2.851258
후임    2.550228
휴가    2.675167
힐링    2.073107
Length: 399, dtype: float64

In [80]:
# tf 데이터 pickle로 저장
with open(fdir+'title_idf.pickle','wb') as f:
    pickle.dump(tf, f, pickle.HIGHEST_PROTOCOL)

In [81]:
# TF-IDF 생성
tf_idf=tf*idf
tf_idf

,가요,가족,가지,갈라,갈만,감,감사,감안,강남,개,...,화요일,환자,활동,회,회사,횡인,후,후임,휴가,힐링
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
# tf 데이터 pickle로 저장
with open(fdir+'title_tfidf.pickle', 'wb') as f:
    pickle.dump(tf_idf, f, pickle.HIGHEST_PROTOCOL)

### question에 대한 TF-IDF 계산하기

In [93]:
# 단어에 대한 고유 목록 생성
vocab=[sorted(list(set([n for qn in question_nouns for n in qn])))]

tmp=["".join(re.findall('[a-zA-Z가-힣0-9]+', w)) for w in title_vocab]
vocab=[w for w in sorted(list(set(tmp))) if w != '']
vocab[:10]

['가요', '가족', '가지', '갈라', '갈만', '감', '감사', '감안', '강남', '개']

In [94]:
# TF 계산
tf_lst=[]
for nrow in question_nouns:
    tf_lst.append([])
    for voc in vocab:
        tf_lst[-1].append(nrow.count(voc))

tf = pd.DataFrame(tf_lst, columns=vocab)
tf

,가요,가족,가지,갈라,갈만,감,감사,감안,강남,개,...,화요일,환자,활동,회,회사,횡인,후,후임,휴가,힐링
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,8,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
# tf 데이터 pickle로 저장
with open(fdir+'question_tf.pickle', 'wb') as f:
    pickle.dump(tf, f, pickle.HIGHEST_PROTOCOL)

In [96]:
# IDF 생성
D = len(tf)
df_ = tf.astype(bool).sum()
idf=np.log10(D/(1+df_))
idf

가요    1.737315
가족    1.110896
가지    2.453318
갈라    3.152288
갈만    1.404100
        ...   
횡인    3.152288
후     1.790561
후임    2.550228
휴가    1.830069
힐링    1.572505
Length: 399, dtype: float64

In [97]:
# tf 데이터 pickle로 저장
with open(fdir+'question_idf.pickle', 'wb') as f:
    pickle.dump(idf, f, pickle.HIGHEST_PROTOCOL)

In [89]:
# IDF 생성
D = len(tf)
df_=tf.astype(bool).sum() # 단어 단위의 빈도수
idf=np.log10(D/(1+df_))
idf

가감     2.675167
가게     2.675167
가격     2.073107
가능     1.809866
가려     1.851258
         ...   
훈련소    2.675167
휴가     1.830069
휴게소    2.453318
희망     2.453318
힐링     1.572505
Length: 1227, dtype: float64

In [90]:
# tf 데이터 pickle로 저장
with open(fdir+'question_idf.pickle','wb') as f:
    pickle.dump(tf, f, pickle.HIGHEST_PROTOCOL)

In [98]:
# TF-IDF 생성
tf_idf=tf*idf
tf_idf

,가요,가족,가지,갈라,갈만,감,감사,감안,강남,개,...,화요일,환자,활동,회,회사,횡인,후,후임,휴가,힐링
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.953631,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.790561,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.953631,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.790561,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,14.640552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1416,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1417,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1418,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [99]:
# tf 데이터 pickle로 저장
with open(fdir+'question_tfidf.pickle', 'wb') as f:
    pickle.dump(tf_idf, f, pickle.HIGHEST_PROTOCOL)

### sklearn을 이용한 TF-IDF 계산

In [100]:
# 원본 데이터 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ques_date       1420 non-null   object
 1   title           1420 non-null   object
 2   question        1420 non-null   object
 3   answer          1420 non-null   object
 4   title_question  1420 non-null   object
dtypes: object(5)
memory usage: 55.6+ KB


In [104]:
from sklearn.feature_extraction.text import CountVectorizer

vect=CountVectorizer() # 공백을 중심으로 단어 분리를 함
doc_term_matrix = vect.fit_transform(df['question'])

tf=pd.DataFrame(doc_term_matrix.toarray(), columns=vect.get_feature_names())
tf

,00,05분,07시쯤,10,100,1003번,100걸어요,100드려요,100드림미다,100만원,...,힐링여행,힐링여행으로,힐링여행을,힐링을,힐링할만한데,힘드네요,힘들,힘들까요,힘들며,힘들어서더
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1417,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
# IDF 계산
D = len(tf)
df=tf.astype(bool).sum(axis=0)
idf=np.log10(D/(df+1))
idf

00       2.675167
05분      2.851258
07시쯤     2.675167
10       2.374137
100      2.110896
           ...   
힘드네요     1.976197
힘들       2.550228
힘들까요     2.851258
힘들며      2.453318
힘들어서더    2.675167
Length: 4665, dtype: float64

In [111]:
# TF-IDF 계싼
tf_idf=tf*idf
tf_idf

,00,05분,07시쯤,10,100,1003번,100걸어요,100드려요,100드림미다,100만원,...,힐링여행,힐링여행으로,힐링여행을,힐링을,힐링할만한데,힘드네요,힘들,힘들까요,힘들며,힘들어서더
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
